<a href="https://colab.research.google.com/github/juliaguardiani/test_ITM_BRASIL/blob/main/pre_procesing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install wandb

In [2]:
!pip install pandas-profiling==3.1.0

In [3]:
import wandb
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from pandas_profiling import ProfileReport
import tempfile
import os

# 1.0 Fetch Data

In [4]:
# importing the dataset
income = pd.read_csv("https://raw.githubusercontent.com/juliaguardiani/test_ITM_BRASIL/main/tabelaFinal.csv")
income.head()

,marca_id,marca,antiss,AV,BA,BC,DRML,dresc,DI,EEG,...,RR_02,RRP_02,SBF_02,SS_02,SMB.1,subc_02,TC_02,TEC_02,TI_02,usuAgua_02
0,1,Adidas,"0,25","0,25","0,25",0,"0,25","0,25","0,25","0,25",...,"0,5","0,5","0,5","0,5","0,5","0,5",0,"0,5","0,5","0,5"
1,2,Animale,"0,25","0,25",0,"0,25","0,25","0,25","0,25","0,25",...,0,"0,5","0,5",0,"0,5",0,0,0,"0,5",0
2,3,Aramis,0,"0,25",0,0,"0,25","0,25","0,25",0,...,0,0,0,"0,5","0,5",0,0,0,0,"0,5"
3,4,Arezzo,"0,25","0,25",0,"0,25",0,"0,25","0,25","0,25",...,"0,5","0,5",0,"0,5","0,5",0,0,0,"0,5",0
4,5,Besni,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
income.to_csv("ITM_BRASIL_2021_data.csv",index=False)

In [ ]:
# Login to Weights & Biases
!wandb login --relogin

In [ ]:
# save_code tracking all changes of the notebook and sync with Wandb
run = wandb.init(project="decision_tree", save_code=True)

In [ ]:
# Send the raw_data.csv to the Wandb storing it as an artifact
!wandb artifact put \
      --name decision_tree/ITM_BRASIL_2021_data.csv \
      --type raw_data \
      --description "ÍNDICE DE TRANSPARÊNCIA DA MODA BRASIL 2021" ITM_BRASIL_2021_data.csv

# 2.0 Pre-procesing

In [9]:
input_artifact="decision_tree/ITM_BRASIL_2021_data.csv:latest"
artifact_name="preprocessed_data.csv"
artifact_type="clean_data"
artifact_description="Data after preprocessing"

In [10]:
# create a new job_type
run = wandb.init(project="decision_tree", job_type="process_data")

In [11]:
# donwload the latest version of artifact raw_data.csv
artifact = run.use_artifact(input_artifact)

# create a dataframe from the artifact
df_total = pd.read_csv(artifact.file())

In [12]:
df_total.head()

,marca_id,marca,antiss,AV,BA,BC,DRML,dresc,DI,EEG,...,RR_02,RRP_02,SBF_02,SS_02,SMB.1,subc_02,TC_02,TEC_02,TI_02,usuAgua_02
0,1,Adidas,"0,25","0,25","0,25",0,"0,25","0,25","0,25","0,25",...,"0,5","0,5","0,5","0,5","0,5","0,5",0,"0,5","0,5","0,5"
1,2,Animale,"0,25","0,25",0,"0,25","0,25","0,25","0,25","0,25",...,0,"0,5","0,5",0,"0,5",0,0,0,"0,5",0
2,3,Aramis,0,"0,25",0,0,"0,25","0,25","0,25",0,...,0,0,0,"0,5","0,5",0,0,0,0,"0,5"
3,4,Arezzo,"0,25","0,25",0,"0,25",0,"0,25","0,25","0,25",...,"0,5","0,5",0,"0,5","0,5",0,0,0,"0,5",0
4,5,Besni,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [13]:
df_total.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 78 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   marca_id    50 non-null     int64 
 1   marca       50 non-null     object
 2   antiss      50 non-null     object
 3   AV          50 non-null     object
 4   BA          50 non-null     object
 5   BC          50 non-null     object
 6   DRML        50 non-null     object
 7   dresc       50 non-null     object
 8   DI          50 non-null     object
 9   EEG         50 non-null     object
 10  EC          50 non-null     object
 11  ES          50 non-null     object
 12  FAFP        50 non-null     object
 13  HTID        50 non-null     object
 14  LADON       50 non-null     object
 15  LSR         50 non-null     object
 16  RE          50 non-null     object
 17  RP          50 non-null     object
 18  SBF         50 non-null     object
 19  SS          50 non-null     object
 20  SMB         

In [14]:
df_total.describe()

,marca_id
count,50.00000
mean,25.50000
std,14.57738
min,1.00000
25%,13.25000
50%,25.50000
75%,37.75000
max,50.00000


In [15]:
total_01 = ['3.5', '3.75', '2', '3.25', '0', '0', '4.75', '0', '0','1.25',
        '0', '0', '4', '2', '2.75', '0.25', '0', '1.5', '1.25', '3.75',
        '0', '3.5', '3.75', '0', '4.5', '2.75', '0', '2.75','0', '0', 
        '0', '3', '4.5', '1.5', '0', '4.5', '0', '0', '0', '0.75', '4',
        '4.25', '0', '3.25', '4', '0', '0', '2.25', '3.25', '4.25']
df_total['total_01'] = total_01 

In [16]:
total_02 = ['8.5', '0', '1', '2.75', '0', '0', '8.75', '0', '0', 
         '0.5', '0', '0', '9', '0.25', '6.5', '0', '0', '1.5', '3.5',
         '0', '0', '8.75', '8.25', '0', '5.25', '4', '0', '4', '0',
         '0', '0', '4.5', '8.75', '7.25', '0', '5.25', '0', '1.5', 
         '0', '0', '7.75', '7', '0', '8', '7.25', '0', '0', '0', '8', '8.5']
df_total['total_02'] = total_02

In [17]:
total_03 = ['14.5', '7.5', '5.5', '6.5', '0.5', '0', '15', '0.5', '0', '0',
         '0', '0', '11', '0', '10', '3', '0', '1', '0.5', '7.5', '0', '8.5',
         '9.5', '2', '10', '3', '0', '3', '0', '0', '0', '8', '12', '2', '0',
         '10', '0', '0.5', '0', '4.5', '10', '10.5', '1', '11.5', '11.5', '0', 
         '0', '0', '11.5', '13.5']
df_total['total_03'] = total_03

In [18]:
for col in df_total.columns:
  df_total[col] = df_total[col].astype(str)
  df_total[col] = df_total[col].str.replace(',','.')

In [19]:
# Generate a "clean data file"
df_total.to_csv(artifact_name,index=False)

In [23]:
df_total.head()

,marca_id,marca,antiss,AV,BA,BC,DRML,dresc,DI,EEG,...,SS_02,SMB.1,subc_02,TC_02,TEC_02,TI_02,usuAgua_02,total_01,total_02,total_03
0,1,Adidas,0.25,0.25,0.25,0,0.25,0.25,0.25,0.25,...,0.5,0.5,0.5,0,0.5,0.5,0.5,3.5,8.5,14.5
1,2,Animale,0.25,0.25,0,0.25,0.25,0.25,0.25,0.25,...,0,0.5,0,0,0,0.5,0,3.75,0,7.5
2,3,Aramis,0,0.25,0,0,0.25,0.25,0.25,0,...,0.5,0.5,0,0,0,0,0.5,2,1,5.5
3,4,Arezzo,0.25,0.25,0,0.25,0,0.25,0.25,0.25,...,0.5,0.5,0,0,0,0.5,0,3.25,2.75,6.5
4,5,Besni,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.5


In [20]:
# Create a new artifact and configure with the necessary arguments
artifact = wandb.Artifact(name=artifact_name,
                          type=artifact_type,
                          description=artifact_description)
artifact.add_file(artifact_name)

<ManifestEntry digest: 2ENpjMZeCj/YInJ8Veuidg==>

In [21]:
# Upload the artifact to Wandb
run.log_artifact(artifact)

In [22]:
run.finish()